粗糙集是波兰理工大学Z.pawlak教授提出用来研究不完整数据，不精确知识的表达、学习，归纳等的一套理论

它是一种新的处理模糊和不确定性问题的数学工具，已被广泛应用于知识发现、机器学习、决策支持、模式识别、专家系统及归纳推理等领域。

粗糙集理论的特点是能够分析隐藏在数据中的事实，又不需要关于数据附加信息。

其主要思想是在保持分类能力不变的前提下，通过知识约简，导出问题的决策或分类规则。

从数学的角度看，粗糙集是研究集合的；从编程的角度看，粗糙集的研究对象是矩阵,只不过是一些特殊的矩阵；从人工智能的角度来看，粗糙集研究的是决策表。
##### 视频讲解粗糙集和属性约简

* [视频讲解粗糙集1](https://www.bilibili.com/video/BV1xu41157ss/?spm_id_from=333.999.0.0&vd_source=ce2c265fe654b727a504cf64875b6105)

* [视频讲解粗糙集2](https://www.bilibili.com/video/BV1D14y1R7qB/?spm_id_from=333.788.recommend_more_video.-1&vd_source=ce2c265fe654b727a504cf64875b6105)

* [视频讲解粗糙集3](https://www.bilibili.com/video/BV18z4y1J7GF/?spm_id_from=333.788.recommend_more_video.0&vd_source=ce2c265fe654b727a504cf64875b6105)
##### 文字讲解什么是粗糙集
* [什么是粗糙集1](https://www.jianshu.com/p/a129b7a6be9e)

* [什么是粗糙集2](https://www.jianshu.com/p/bb61d6bc81e1)

* [什么是粗糙集3](https://www.jianshu.com/p/ab3135fd5d40)

* [什么是粗糙集4](https://www.jianshu.com/p/b48d46688078)

* [什么是粗糙集5](https://www.jianshu.com/p/768166d73aa1)

* [什么是粗糙集6](https://www.jianshu.com/p/b303f95313cf)

In [173]:
import numpy as np
import pandas as pd


In [174]:
table = pd.DataFrame(
    data=np.matrix([["e1",'是', '是', '正常', '否']
    ,["e2",'是', '是', '高', '是']
    ,["e3",'是', '是', '很高', '是']
    ,["e4",'否', '是', '正常', '否']
    ,["e5",'否', '否', '高', '否']
    ,["e6",'否', '是', '很高', '是']])
    ,columns= ["病人","头疼","肌肉疼","体温","流感"]
    # ,index = ['e1','e2','e3','e4','e5','e6']
        )
table

,病人,头疼,肌肉疼,体温,流感
0,e1,是,是,正常,否
1,e2,是,是,高,是
2,e3,是,是,很高,是
3,e4,否,是,正常,否
4,e5,否,否,高,否
5,e6,否,是,很高,是


#### 决策属性 $ D $
* 决策属性其实就相当于平时分类建模时候训练数据的标签
* 在这里决策属性其实就是 “流感” 这一列

In [175]:
D = np.ravel(table.iloc[:,-1])
D = np.array(D,dtype=np.str_)

#### 条件属性 $ C $
* 条件属性就是去掉索引和标签的部分

In [176]:
C = np.matrix(table.iloc[:,1:-1],dtype=np.str_)
C 

matrix([['是', '是', '正常'],
        ['是', '是', '高'],
        ['是', '是', '很高'],
        ['否', '是', '正常'],
        ['否', '否', '高'],
        ['否', '是', '很高']], dtype='<U2')

#### $ R $ 其实就是个属性值的列名

In [177]:
R = np.array(table.columns,dtype=np.str_)
R

array(['病人', '头疼', '肌肉疼', '体温', '流感'], dtype='<U3')

#### 论域是一个对象$ U $
* 论域其实就是平时建模时候的 X 变量矩阵
* 在信息系统中，这就是 U ，是非空有限对象集，称为论域
* $ U = \{e1,e2,e3,e4,e5,e6\} $ [e1,e2 ……]这些其实是论域的标签

In [178]:
U = np.matrix(table.iloc[:,:],dtype=np.str_)
U

matrix([['e1', '是', '是', '正常', '否'],
        ['e2', '是', '是', '高', '是'],
        ['e3', '是', '是', '很高', '是'],
        ['e4', '否', '是', '正常', '否'],
        ['e5', '否', '否', '高', '否'],
        ['e6', '否', '是', '很高', '是']], dtype='<U2')

####  $ V $ 是属性值的集合

In [179]:
V = np.unique(np.ravel(U[:,1:]))
V

array(['否', '很高', '是', '正常', '高'], dtype='<U2')

#### $ V_{a} $ 就是各属性的值域，白话说就是各属性的值的取值范围
##### 比如这张表里:
* $ V_{头疼} $ ：的取值范围就是 {是，否}
* $ V_{肌肉疼 } $ ： 的取值范围就是 {是，否}
* $ V_{体温} $ ： 的取值范围就是 {正常，高，很高}

In [180]:
def GetVa(U,R):
    return_eq = {"变量名":[],"值域":[]}
    # 切分出值矩阵
    Vmatrix = U[:,1:]
    # 且分出列明
    Rs = R[1:]

    for R_name,Varray in zip(Rs,Vmatrix):
        # 先将列数据降维到行数据
        lower_dimensional = np.ravel(Varray)
        # 对数据去重得到 Va 也就是属性的值域
        Va = np.unique(lower_dimensional) 
        return_eq["变量名"].append(R_name)
        return_eq["值域"].append(Va)

    return pd.DataFrame(return_eq)

GetVa(U,R)

,变量名,值域
0,头疼,"[否, 是, 正常]"
1,肌肉疼,"[是, 高]"
2,体温,"[很高, 是]"
3,流感,"[否, 是, 正常]"


$f$ : 是 $U \times R \rightarrow V$ 的一个信息函数, 它为每个对象 $\mathbf{x}$ 的 每个属性 $\mathbf{a}$ 赋予一个属性值, 即
$$
a \in R, x \in U, f_a(x) \in V_a
$$
* 比如 $ e1 $ 在这里就是 【头疼】这一项 映射的值就是【是】，这个过程就是这个信息函数的作用  
* 这里调用f函数的要求是： $ x \in U $ $ a \in R $ 
* 返回的 $f(x) \in V$

In [181]:
def f(U,R,x,a):
    '''
    信息函数
    '''
    a = np.array(a)
    x = np.array(x)
    # 切分出 U 的标签 e_{x}
    U_i = U[:,0]

    # 生成 U 轴的布尔值索引
    m,n = U_i.shape
    x_index = np.zeros(m)
    for x_i in x: 
        x_index += np.ravel(U_i==x_i)
    # 将x所属U的位置数字变为bool值索引 
    x_index = x_index.astype(bool)

    # 生成 R 轴的布尔值索引
    m = R.size
    a_index = np.zeros(m)
    for a_i in a:
        a_index += np.ravel(R == a_i)
    # 将a所属R的位置数字变为bool值索引 
    a_index = a_index.astype(bool)

    # 切分出当前信息
    fx = U[x_index].T[a_index].T
    return pd.DataFrame(data=fx,columns=a,index=x)

f(U,R,a=["头疼","体温"],x=["e1","e2"])

,头疼,体温
e1,是,正常
e2,是,高


#### IND(A) 等价关系
* 大白话说等价关系就是两或者多个对象,在某一个或着多个属性上相同，那么就说他们之间是等价关系
* A 可能包含一个属性，或者多个属性

In [213]:
def IND(U,R,A:iter,out_dataframe=None):
    # 切分出 U 的标签 e_{x}
    U_i = U[:,0]
    m = R.size
    A = np.array(A)
    a_index = np.zeros(m)
    for a_i in A:
        a_index += np.ravel(R == a_i)
    # 将a所属R的位置数字变为bool值索引 
    a_index = a_index.astype(bool)

    # 切出待比较的列
    U_ij = U.T[a_index].T
    # 去重相同属性
    de_duplicate_Uij = np.unique(U_ij,axis=0)


    A_length = len(A)
    return_eq = {"比较属性列名":[],"等价属性":[],"等价类对象":[]}
    for de_ in de_duplicate_Uij:
        x_index = np.where(U_ij == de_,True,False).sum(axis=1)==len(A)
        return_eq["比较属性列名"].append(A)
        return_eq["等价属性"].append(de_)
        return_eq["等价类对象"].append(np.ravel(U_i[x_index]))
        # print(de_,np.ravel(U_i[x_index]))
    if out_dataframe:
        return pd.DataFrame(return_eq)
    return return_eq

IND(U,R,A=["肌肉疼"],out_dataframe=True)

,比较属性列名,等价属性,等价类对象
0,[肌肉疼],[否],[e5]
1,[肌肉疼],[是],"[e1, e2, e3, e4, e6]"
